In [1]:
import os
import h5py
import tensorflow as tf
import keras
import pandas as pd
import shutil
import numpy as np
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense, Flatten, Dropout
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.pooling import GlobalAveragePooling2D
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

C:\Users\Zhiye\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [2]:
###Setting batch size and define path to saved model
batch_size = 80
top_model_weights_path = 'VGG19.h5'

In [3]:
#A function to extract and save bottleneck features of VGG19 model. It also saves x_train, y_train, x_val, y_val to npy files.
def save_bottleneck_features():
    
    #information to create train and validation data. Generating one hot labels.
    train_DF = pd.read_csv('../../../labels.csv')
    filenames = pd.Series(train_DF['id'])
    series = pd.Series(train_DF['breed'])
    unique_breeds = np.unique(series)
    filenames_train = []
    filenames_validate = []
    one_hot_labels = np.asarray(pd.get_dummies(series, sparse = True))
    
    # Creating and copying data to train and validation folder so that we can use ImageDataGenerator later.
    for i in tqdm(range(len(filenames))):
        label = unique_breeds[np.where(one_hot_labels[i]==1.)][0]
        filename = '{}.jpg'.format(filenames[i])

        if i < 8000:
            new_dir = '../../../sorted/train/{}/'.format(label)
            filenames_train.append(new_dir + filename)
        else:
            new_dir = '../../../sorted/validate/{}/'.format(label)
            filenames_validate.append(new_dir + filename)
        if not os.path.exists(new_dir):
            os.makedirs(new_dir)
    
        shutil.copy("../../../train/{}.jpg".format(filenames[i]), new_dir + filename)

    #sorting train and validation data/labels
    indices_train = np.argsort(filenames_train)
    indices_val = np.argsort(filenames_validate)
    sorted_filenames_train = np.array(filenames_train)[indices_train]
    sorted_filenames_validate = np.array(filenames_validate)[indices_val]
    sorted_labels_train = np.array(one_hot_labels)[0:8000][indices_train]
    sorted_labels_validate = np.array(one_hot_labels)[8000:][indices_val]
        
    # creating image data generator
    train_datagen = ImageDataGenerator()
    val_datagen = ImageDataGenerator()
    
    #load pre_trained vgg19 model
    vgg19 = keras.applications.vgg19.VGG19(include_top=False, weights='imagenet',input_tensor=None, input_shape=(224,224,3))
    
    
    #generating and saving x_train, y_train, x_val, y_val to npy files.
    train_gen = train_datagen.flow_from_directory("../../../sorted/train", 
                                              batch_size=batch_size, 
                                              target_size=(224, 224), 
                                              shuffle=False)

    x_train = vgg19.predict_generator(train_gen, 
                                  steps=8000 / batch_size, 
                                  verbose=1)
    
    np.save(open('x_train.npy', 'wb'),x_train)
    
    val_gen = val_datagen.flow_from_directory("../../../sorted/validate", 
                                          batch_size=batch_size, 
                                          target_size=(224, 224), 
                                          shuffle=False)
    
    x_val = vgg19.predict_generator(val_gen, 
                                steps=2222 / batch_size, 
                                verbose=1)
    
    np.save(open('x_val.npy', 'wb'),x_val)
    
    
    y_train = sorted_labels_train[0:len(x_train)]
    np.save(open('y_train.npy', 'wb'),y_train)
    
    y_val = sorted_labels_validate[0:len(x_val)]
    np.save(open('y_val.npy', 'wb'),y_val)
    
    return vgg19


In [4]:
vgg19 = save_bottleneck_features()

100%|███████████████████████████████████████████████████████████████████████████| 10222/10222 [00:15<00:00, 666.74it/s]


Found 8000 images belonging to 120 classes.
100/100 [==============================] - ETA: 3: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 57s - ETA: 52 - ETA: 48 - ETA: 45 - ETA: 43 - ETA: 40 - ETA: 39 - ETA: 37 - ETA: 36 - ETA: 35 - ETA: 33 - ETA: 32 - ETA: 32 - ETA: 31 - ETA: 30 - ETA: 29 - ETA: 29 - ETA: 28 - ETA: 27 - ETA: 27 - ETA: 26 - ETA: 26 - ETA: 25 - ETA: 24 - ETA: 24 - ETA: 24 - ETA: 23 - ETA: 23 - ETA: 22 - ETA: 22 - ETA: 21 - ETA: 21 - ETA: 20 - ETA: 20 - ETA: 20 - ETA: 19 - ETA: 19 - ETA: 18 - ETA: 18 - ETA: 18 - ETA: 17 - ETA: 17 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  -

In [5]:
def train_model(vgg19,loadWeight):
    #loading x_train, y_train, x_val, y_val 
    x_train = np.load('x_train.npy')
    y_train = np.load('y_train.npy')
    x_val = np.load('x_val.npy')
    y_val = np.load('y_val.npy') 

    # Turn off training vgg19
    for layer in vgg19.layers:
        layer.trainable = False
    
    #To get fully connected layers
    vgg19_full = keras.applications.vgg19.VGG19(include_top=True, weights='imagenet',input_tensor=None, input_shape=(224,224,3))
    
    # Turn off training for these two fully connected layers from vgg19
    fc1_layer = vgg19_full.get_layer("fc1")
    fc1_layer.trainable = False
    fc2_layer = vgg19_full.get_layer("fc2")
    fc2_layer.trainable = False  
    
    #Model
    inputs = Input(shape=(7,7,512))
    x = Flatten()(inputs)
    x = fc1_layer(x)
    x = BatchNormalization()(x)
    x = Dropout(0.8)(x)
    x = fc2_layer(x)
    x = BatchNormalization()(x)
    x = Dropout(0.8)(x)     
    x = Dense(1028, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.8)(x)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.8)(x)
    x = Dense(120, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=x)
    model.summary()
    
    #check if load existing weights.
    if(loadWeight == True):
        model.load_weights(top_model_weights_path)
        return model
    
    
    #Training model
    model.compile(optimizer=keras.optimizers.Adam(), 
              loss=keras.losses.categorical_crossentropy, 
              metrics=['accuracy'])
    
    model.fit(x_train, y_train, batch_size=128, epochs=1000, verbose=1, 
                    validation_data=(x_val, y_val))

    #saving weights
    model.save_weights(top_model_weights_path)
    return model

In [6]:
model = train_model(vgg19,False)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 7, 7, 512)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
fc1 (Dense)                  (None, 4096)              102764544 
_________________________________________________________________
batch_normalization_1 (Batch (None, 4096)              16384     
_________________________________________________________________
dropout_1 (Dropout)          (None, 4096)              0         
_________________________________________________________________
fc2 (Dense)                  (None, 4096)              16781312  
_________________________________________________________________
batch_normalization_2 (Batch (None, 4096)              16384     
__________

8000/8000 [==============================] - ETA: 2s - loss: 2.6846 - acc: 0.257 - ETA: 1s - loss: 2.9748 - acc: 0.250 - ETA: 1s - loss: 3.0426 - acc: 0.225 - ETA: 1s - loss: 3.0794 - acc: 0.224 - ETA: 1s - loss: 3.0630 - acc: 0.228 - ETA: 1s - loss: 3.0361 - acc: 0.233 - ETA: 1s - loss: 3.0541 - acc: 0.223 - ETA: 1s - loss: 3.0758 - acc: 0.223 - ETA: 1s - loss: 3.0658 - acc: 0.224 - ETA: 1s - loss: 3.0636 - acc: 0.226 - ETA: 1s - loss: 3.0735 - acc: 0.225 - ETA: 1s - loss: 3.0723 - acc: 0.225 - ETA: 1s - loss: 3.0837 - acc: 0.221 - ETA: 1s - loss: 3.0613 - acc: 0.226 - ETA: 0s - loss: 3.0603 - acc: 0.226 - ETA: 0s - loss: 3.0497 - acc: 0.223 - ETA: 0s - loss: 3.0455 - acc: 0.222 - ETA: 0s - loss: 3.0469 - acc: 0.222 - ETA: 0s - loss: 3.0454 - acc: 0.221 - ETA: 0s - loss: 3.0397 - acc: 0.222 - ETA: 0s - loss: 3.0353 - acc: 0.222 - ETA: 0s - loss: 3.0339 - acc: 0.221 - ETA: 0s - loss: 3.0386 - acc: 0.219 - ETA: 0s - loss: 3.0423 - acc: 0.218 - ETA: 0s - loss: 3.0382 - acc: 0.217 - ETA: 

8000/8000 [==============================] - ETA: 1s - loss: 2.4761 - acc: 0.242 - ETA: 1s - loss: 2.5810 - acc: 0.273 - ETA: 1s - loss: 2.5215 - acc: 0.293 - ETA: 1s - loss: 2.5182 - acc: 0.297 - ETA: 1s - loss: 2.5099 - acc: 0.296 - ETA: 1s - loss: 2.5158 - acc: 0.297 - ETA: 1s - loss: 2.5342 - acc: 0.295 - ETA: 1s - loss: 2.5305 - acc: 0.299 - ETA: 1s - loss: 2.5275 - acc: 0.302 - ETA: 1s - loss: 2.5197 - acc: 0.303 - ETA: 0s - loss: 2.5282 - acc: 0.303 - ETA: 0s - loss: 2.5282 - acc: 0.304 - ETA: 0s - loss: 2.5242 - acc: 0.305 - ETA: 0s - loss: 2.5291 - acc: 0.304 - ETA: 0s - loss: 2.5386 - acc: 0.301 - ETA: 0s - loss: 2.5383 - acc: 0.302 - ETA: 0s - loss: 2.5386 - acc: 0.303 - ETA: 0s - loss: 2.5343 - acc: 0.304 - ETA: 0s - loss: 2.5433 - acc: 0.306 - ETA: 0s - loss: 2.5540 - acc: 0.301 - ETA: 0s - loss: 2.5573 - acc: 0.302 - ETA: 0s - loss: 2.5580 - acc: 0.301 - ETA: 0s - loss: 2.5564 - acc: 0.301 - ETA: 0s - loss: 2.5585 - acc: 0.302 - ETA: 0s - loss: 2.5691 - acc: 0.299 - ETA: 

8000/8000 [==============================] - ETA: 1s - loss: 2.1942 - acc: 0.414 - ETA: 2s - loss: 2.3375 - acc: 0.402 - ETA: 2s - loss: 2.3698 - acc: 0.384 - ETA: 2s - loss: 2.3767 - acc: 0.378 - ETA: 1s - loss: 2.4030 - acc: 0.362 - ETA: 1s - loss: 2.4133 - acc: 0.354 - ETA: 1s - loss: 2.4236 - acc: 0.354 - ETA: 1s - loss: 2.4148 - acc: 0.358 - ETA: 1s - loss: 2.4304 - acc: 0.353 - ETA: 1s - loss: 2.4284 - acc: 0.355 - ETA: 1s - loss: 2.4281 - acc: 0.349 - ETA: 1s - loss: 2.4194 - acc: 0.347 - ETA: 1s - loss: 2.4091 - acc: 0.345 - ETA: 1s - loss: 2.4028 - acc: 0.345 - ETA: 1s - loss: 2.4064 - acc: 0.344 - ETA: 1s - loss: 2.3929 - acc: 0.345 - ETA: 0s - loss: 2.3938 - acc: 0.345 - ETA: 0s - loss: 2.4018 - acc: 0.344 - ETA: 0s - loss: 2.3985 - acc: 0.342 - ETA: 0s - loss: 2.3972 - acc: 0.342 - ETA: 0s - loss: 2.3950 - acc: 0.341 - ETA: 0s - loss: 2.3998 - acc: 0.339 - ETA: 0s - loss: 2.4074 - acc: 0.339 - ETA: 0s - loss: 2.4081 - acc: 0.339 - ETA: 0s - loss: 2.4061 - acc: 0.339 - ETA: 

8000/8000 [==============================] - ETA: 1s - loss: 2.2533 - acc: 0.429 - ETA: 1s - loss: 2.2581 - acc: 0.386 - ETA: 1s - loss: 2.2787 - acc: 0.375 - ETA: 1s - loss: 2.2893 - acc: 0.375 - ETA: 1s - loss: 2.2753 - acc: 0.378 - ETA: 1s - loss: 2.2781 - acc: 0.375 - ETA: 1s - loss: 2.3030 - acc: 0.370 - ETA: 1s - loss: 2.3185 - acc: 0.365 - ETA: 1s - loss: 2.3265 - acc: 0.359 - ETA: 0s - loss: 2.3120 - acc: 0.358 - ETA: 0s - loss: 2.3224 - acc: 0.356 - ETA: 0s - loss: 2.3179 - acc: 0.359 - ETA: 0s - loss: 2.3230 - acc: 0.359 - ETA: 0s - loss: 2.3233 - acc: 0.361 - ETA: 0s - loss: 2.3215 - acc: 0.360 - ETA: 0s - loss: 2.3150 - acc: 0.359 - ETA: 0s - loss: 2.3233 - acc: 0.360 - ETA: 0s - loss: 2.3187 - acc: 0.363 - ETA: 0s - loss: 2.3265 - acc: 0.361 - ETA: 0s - loss: 2.3240 - acc: 0.361 - ETA: 0s - loss: 2.3190 - acc: 0.363 - ETA: 0s - loss: 2.3232 - acc: 0.362 - ETA: 0s - loss: 2.3303 - acc: 0.360 - ETA: 0s - loss: 2.3279 - acc: 0.361 - ETA: 0s - loss: 2.3291 - acc: 0.360 - ETA: 

...

8000/8000 [==============================] - ETA: 1s - loss: 1.7357 - acc: 0.546 - ETA: 1s - loss: 1.7586 - acc: 0.531 - ETA: 1s - loss: 1.7376 - acc: 0.517 - ETA: 1s - loss: 1.8003 - acc: 0.497 - ETA: 1s - loss: 1.8072 - acc: 0.489 - ETA: 1s - loss: 1.7930 - acc: 0.490 - ETA: 1s - loss: 1.8041 - acc: 0.489 - ETA: 1s - loss: 1.8022 - acc: 0.494 - ETA: 1s - loss: 1.8140 - acc: 0.488 - ETA: 1s - loss: 1.8361 - acc: 0.482 - ETA: 1s - loss: 1.8602 - acc: 0.476 - ETA: 1s - loss: 1.8532 - acc: 0.475 - ETA: 1s - loss: 1.8557 - acc: 0.476 - ETA: 0s - loss: 1.8519 - acc: 0.475 - ETA: 0s - loss: 1.8499 - acc: 0.474 - ETA: 0s - loss: 1.8470 - acc: 0.475 - ETA: 0s - loss: 1.8468 - acc: 0.474 - ETA: 0s - loss: 1.8470 - acc: 0.472 - ETA: 0s - loss: 1.8385 - acc: 0.474 - ETA: 0s - loss: 1.8350 - acc: 0.475 - ETA: 0s - loss: 1.8365 - acc: 0.474 - ETA: 0s - loss: 1.8336 - acc: 0.475 - ETA: 0s - loss: 1.8362 - acc: 0.476 - ETA: 0s - loss: 1.8351 - acc: 0.475 - ETA: 0s - loss: 1.8360 - acc: 0.475 - ETA: 

8000/8000 [==============================] - ETA: 1s - loss: 1.8751 - acc: 0.421 - ETA: 1s - loss: 1.8314 - acc: 0.470 - ETA: 1s - loss: 1.8427 - acc: 0.476 - ETA: 1s - loss: 1.8378 - acc: 0.484 - ETA: 1s - loss: 1.8719 - acc: 0.478 - ETA: 1s - loss: 1.8438 - acc: 0.476 - ETA: 1s - loss: 1.8324 - acc: 0.481 - ETA: 1s - loss: 1.8242 - acc: 0.481 - ETA: 1s - loss: 1.8055 - acc: 0.483 - ETA: 1s - loss: 1.8090 - acc: 0.485 - ETA: 1s - loss: 1.8096 - acc: 0.486 - ETA: 0s - loss: 1.8240 - acc: 0.483 - ETA: 0s - loss: 1.8206 - acc: 0.481 - ETA: 0s - loss: 1.8325 - acc: 0.477 - ETA: 0s - loss: 1.8377 - acc: 0.475 - ETA: 0s - loss: 1.8335 - acc: 0.479 - ETA: 0s - loss: 1.8425 - acc: 0.475 - ETA: 0s - loss: 1.8409 - acc: 0.477 - ETA: 0s - loss: 1.8355 - acc: 0.478 - ETA: 0s - loss: 1.8282 - acc: 0.480 - ETA: 0s - loss: 1.8242 - acc: 0.481 - ETA: 0s - loss: 1.8279 - acc: 0.478 - ETA: 0s - loss: 1.8326 - acc: 0.478 - ETA: 0s - loss: 1.8389 - acc: 0.477 - ETA: 0s - loss: 1.8365 - acc: 0.478 - ETA: 

8000/8000 [==============================] - ETA: 1s - loss: 1.8914 - acc: 0.445 - ETA: 1s - loss: 1.7620 - acc: 0.486 - ETA: 1s - loss: 1.7656 - acc: 0.468 - ETA: 1s - loss: 1.7968 - acc: 0.478 - ETA: 1s - loss: 1.8237 - acc: 0.474 - ETA: 1s - loss: 1.8293 - acc: 0.475 - ETA: 1s - loss: 1.8087 - acc: 0.477 - ETA: 1s - loss: 1.8093 - acc: 0.473 - ETA: 1s - loss: 1.8175 - acc: 0.468 - ETA: 1s - loss: 1.8194 - acc: 0.472 - ETA: 1s - loss: 1.8271 - acc: 0.469 - ETA: 1s - loss: 1.8260 - acc: 0.470 - ETA: 1s - loss: 1.8167 - acc: 0.470 - ETA: 0s - loss: 1.8228 - acc: 0.470 - ETA: 0s - loss: 1.8204 - acc: 0.473 - ETA: 0s - loss: 1.8180 - acc: 0.473 - ETA: 0s - loss: 1.8197 - acc: 0.476 - ETA: 0s - loss: 1.8140 - acc: 0.478 - ETA: 0s - loss: 1.8134 - acc: 0.479 - ETA: 0s - loss: 1.8145 - acc: 0.480 - ETA: 0s - loss: 1.8223 - acc: 0.479 - ETA: 0s - loss: 1.8169 - acc: 0.481 - ETA: 0s - loss: 1.8154 - acc: 0.482 - ETA: 0s - loss: 1.8167 - acc: 0.481 - ETA: 0s - loss: 1.8181 - acc: 0.480 - ETA: 

8000/8000 [==============================] - ETA: 1s - loss: 1.6076 - acc: 0.562 - ETA: 1s - loss: 1.8913 - acc: 0.494 - ETA: 1s - loss: 1.9139 - acc: 0.476 - ETA: 1s - loss: 1.8323 - acc: 0.490 - ETA: 1s - loss: 1.8590 - acc: 0.479 - ETA: 1s - loss: 1.8370 - acc: 0.485 - ETA: 1s - loss: 1.8414 - acc: 0.483 - ETA: 1s - loss: 1.8512 - acc: 0.479 - ETA: 1s - loss: 1.8388 - acc: 0.483 - ETA: 1s - loss: 1.8454 - acc: 0.481 - ETA: 1s - loss: 1.8484 - acc: 0.483 - ETA: 1s - loss: 1.8460 - acc: 0.481 - ETA: 1s - loss: 1.8334 - acc: 0.480 - ETA: 0s - loss: 1.8321 - acc: 0.477 - ETA: 0s - loss: 1.8277 - acc: 0.476 - ETA: 0s - loss: 1.8312 - acc: 0.475 - ETA: 0s - loss: 1.8276 - acc: 0.475 - ETA: 0s - loss: 1.8267 - acc: 0.474 - ETA: 0s - loss: 1.8246 - acc: 0.474 - ETA: 0s - loss: 1.8312 - acc: 0.470 - ETA: 0s - loss: 1.8358 - acc: 0.468 - ETA: 0s - loss: 1.8369 - acc: 0.466 - ETA: 0s - loss: 1.8329 - acc: 0.468 - ETA: 0s - loss: 1.8311 - acc: 0.468 - ETA: 0s - loss: 1.8313 - acc: 0.467 - ETA: 

8000/8000 [==============================] - ETA: 1s - loss: 1.7470 - acc: 0.523 - ETA: 1s - loss: 1.6961 - acc: 0.505 - ETA: 1s - loss: 1.7519 - acc: 0.490 - ETA: 1s - loss: 1.7684 - acc: 0.497 - ETA: 1s - loss: 1.8046 - acc: 0.482 - ETA: 1s - loss: 1.8230 - acc: 0.474 - ETA: 1s - loss: 1.8167 - acc: 0.477 - ETA: 1s - loss: 1.8332 - acc: 0.471 - ETA: 1s - loss: 1.8249 - acc: 0.475 - ETA: 1s - loss: 1.8315 - acc: 0.477 - ETA: 1s - loss: 1.8322 - acc: 0.476 - ETA: 1s - loss: 1.8367 - acc: 0.473 - ETA: 0s - loss: 1.8350 - acc: 0.474 - ETA: 0s - loss: 1.8358 - acc: 0.474 - ETA: 0s - loss: 1.8341 - acc: 0.475 - ETA: 0s - loss: 1.8276 - acc: 0.476 - ETA: 0s - loss: 1.8208 - acc: 0.480 - ETA: 0s - loss: 1.8247 - acc: 0.478 - ETA: 0s - loss: 1.8197 - acc: 0.480 - ETA: 0s - loss: 1.8235 - acc: 0.477 - ETA: 0s - loss: 1.8316 - acc: 0.476 - ETA: 0s - loss: 1.8326 - acc: 0.475 - ETA: 0s - loss: 1.8478 - acc: 0.473 - ETA: 0s - loss: 1.8470 - acc: 0.473 - ETA: 0s - loss: 1.8488 - acc: 0.474 - ETA: 

In [ ]:
#loading test images
from keras.preprocessing import image
original_test_dir = '../../../test'
def load_test_image(fpath):
    img = image.load_img(fpath, target_size=(224, 224))
    x = image.img_to_array(img)
    return x

test_labels = np.loadtxt('../../../sample_submission.csv', delimiter=',', dtype=str, skiprows=1)
test_images = []
test_names = test_labels[:,0]
for test_name in test_names:
    fname = '{}.jpg'.format(test_name)
    data = load_test_image(os.path.join(original_test_dir, fname))
    test_images.append(data)

test_images = np.asarray(test_images)
test_images = test_images.astype('float32')
print(test_images.shape)

In [ ]:
#using vgg19 to predict
predictions = vgg19.predict(test_images, verbose=1)

In [ ]:
#use the output of vgg19 to further predict final results
predictions = model.predict(predictions, verbose=1)

In [ ]:
#generating Kaggle upload file
df_train = pd.read_csv('../../../labels.csv')
targets_series = pd.Series(df_train['breed'])
one_hot = pd.get_dummies(targets_series, sparse = True)
one_hot_labels = np.asarray(one_hot)

In [ ]:
#generating Kaggle upload file
test_DF = pd.read_csv('../../../sample_submission.csv')
sub = pd.DataFrame(predictions)
col_names = one_hot.columns.values
sub.columns = col_names
sub.insert(0, 'id', test_DF['id'])
sub.head(5)
print(sub.shape)
sub.to_csv('out19_5.csv', sep=',',index=False)

In [ ]:
vgg19.summary()